# Integrate Modern Data Architectures with Generative AI and Interact Using Prompts for Querying SQL Databases and APIs

This notebook demonstrates how large language models (LLMs) that are accessible through [Amazon Bedrock](https://aws.amazon.com/bedrock/), such as Amazon Titan, interact with AWS databases, data stores, and third-party data warehousing solutions, such as [Amazon Athena](https://aws.amazon.com/athena/features/). This interaction is showcased by generating and running SQL queries, and by making requests to API endpoints. The LangChain framework, used to accomplish this demonstration, allows an LLM to interact with its environment and connect with other sources of data. The LangChain framework operates based on three principles: calling out to a language model, being data-aware, and being agentic. 

This notebook focuses on establishing a connection to one data source, consolidating metadata, and returning fact-based data points in response to user queries by using LLMs and LangChain. The solution can be enhanced to add multiple data sources.


<img src='img-genai-sql-langchain-overall-solution.png' width="800" height="600">


### Prerequisites:
1. Use the Python 3 (Data Science 3.0) kernel.
2. Install the required packages.
3. Run the one-time setup by entering the user input parameters, copying the dataset, and running the AWS Glue crawler.
3. Access to the LLM. The Amazon Titan Lite model is used in this notebook. For more information, see Amazon Titan Text models in the Amazon Bedrock User Guide at https://docs.aws.amazon.com/bedrock/latest/userguide/titan-text-models.html.



### Solution walkthrough:

Step 1. Read into a pandas DataFrame the library data JSON file that was downloaded from Amazon S3.

Step 2. Populate the AWS Glue Data Catalog by running an AWS Glue crawler on the staged JSON file. 

Step 3. To obtain information on the data schema, use the SQLAlchemy library to query the Data Catalog.

Step 4. Define the functions to determine the best data channel to answer the user query, and then generate a response to the user query as a correctly formatted SQL statement.

Step 5. Prompt the LLM through LangChain to determine the data channel. After determining the data channel, run the Langchain SQL database chain to convert 'text to sql', and then run the query against the source data channel. 

Finally, display the results.

## Code cell 1 ##


In [ ]:
%%capture

!pip install sqlalchemy==2.0.29
!pip install langchain==0.1.19
!pip install langchain-experimental==0.0.58
!pip install PyAthena[SQLAlchemy]==3.8.2
!pip install -U langchain-aws==0.1.3

## Code cell 2 ##

In [ ]:
import os
import json
import boto3

import sqlalchemy
from sqlalchemy import create_engine

from langchain.docstore.document import Document
from langchain import PromptTemplate,SagemakerEndpoint,SQLDatabase,LLMChain
from langchain_experimental.sql import SQLDatabaseChain, SQLDatabaseSequentialChain
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts.prompt import PromptTemplate

from langchain.chains.api.prompt import API_RESPONSE_PROMPT
from langchain.chains import APIChain

from typing import Dict
import time
from langchain_aws import BedrockLLM

### Library dataset
The dataset for this solution was originally stored in an Amazon Simple Storage Service (Amazon S3) bucket and downloaded locally as the file, s3_library_data.json. 

The dataset consists of the following columns:
* book_id
* title
* author
* genre
* pub_date

You can use the following commands to load the JSON data into a pandas DataFrame and view the data.

## Code cell 3 ##

In [ ]:
import pandas as pd
library_df = pd.read_json('s3_library_data.json',lines=True)
library_df

## One-time setup
### AWS CloudFormation outputs

Some of the resources needed for this notebook have already been created for you as part of lab deployment. These preprovisioned resources include an S3 bucket to store data, AWS Glue databases, and AWS Glue crawlers. 

In order to proceed with the process, you first need to set some variables:
* Lab files bucket
* Library database name
* Library crawler name

Note: Make sure to modify the following CloudFormation stack name.

## Code cell 4 ##


In [ ]:
## Update with the correct CloudFormation stack name.
cloudformation_stack_name = '<LabStack>'
cfn_client = boto3.client('cloudformation')

def get_cfn_outputs(cloudformation_stack_name):
    outputs = {}
    for output in cfn_client.describe_stacks(StackName=cloudformation_stack_name)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

outputs = get_cfn_outputs(cloudformation_stack_name)
json_formatted_str = json.dumps(outputs, indent=2)
print(json_formatted_str)

## Code cell 5 ##

In [ ]:
# The following variables define the AWS Glue Data Catalog database and crawler to use.

## DIY note ##  You must update some of the following variables as part of the later DIY section of this solution.
data_file = 's3_library_data.json'
lab_files_folder = 'library-data'
glue_db_name = outputs['LibraryDatabaseName']
glue_crawler_name = outputs['LibraryCrawlerName']

# The following variable does not need to be changed.
lab_files_bucket = outputs['LabfilesBucketName']

print(lab_files_bucket)

### Stage data

Use the following command to copy the library JSON data to the S3 bucket's library-data folder.

Note: In the later DIY section, you must modify some of the previously initialized variables.

## Code cell 6 ##

In [ ]:
# The following command does not need to be changed.

!aws s3 cp {data_file} s3://{lab_files_bucket}/{lab_files_folder}/

### Run the AWS Glue crawler

The crawler runs and crawls the specified S3 bucket and folder. The crawler then creates tables based on the data found. The crawled data is available for querying through Amazon Athena, or nontechnical users can use the power of LLMs to convert text questions into SQL.

## Code cell 7 ##

In [ ]:
client = boto3.client('glue')

print("About to start running the crawler: ", glue_crawler_name)

try:
    response = client.start_crawler(Name=glue_crawler_name )
    print("Successfully started crawler. The crawler may take 2-5 mins to detect the schema.")
    while True:
        # Get the crawler status.
        response = client.get_crawler(Name=glue_crawler_name)
         # Extract the crawler state.
        status = response['Crawler']['State']
        # Print the crawler status.
        print(f"Crawler '{glue_crawler_name}' status: {status}")
        if status == 'STOPPING':  # Replace 'READY' with the desired completed state.
            break  # Exit the loop if the desired state is reached.

        time.sleep(10)  # Sleep for 10 seconds before checking the status again.
    
except:
    print("error in starting crawler. Check the logs for the error details.")

Note: Before proceeding to the next step, check to confirm that the crawler status changed from RUNNING to STOPPING.

### Step 1 - Connect to databases by using SQLAlchemy. 

LangChain uses SQLAlchemy to connect to SQL databases. SQLDatabaseChain can be used with any SQL dialect supported by SQLAlchemy, such as MS SQL, MySQL, MariaDB, PostgreSQL, Oracle SQL, and SQLite. For more information about requirements for connecting to your database, see the SQLAlchemy documentation. 

Note: The following code establishes a database connection for data sources and LLMs. Note that the solution will work only if the database connection for your sources is defined in the next code cell. For more information, see the previous Prerequisites section.

## Code cell 8 ##

In [ ]:
# Define the AWS Region.
region = boto3.session.Session().region_name

## Athena variables
connathena=f"athena.{region}.amazonaws.com" 
portathena='443'                                         # Update, if port is different
schemaathena=glue_db_name                                # from user defined params
s3stagingathena=f's3://{lab_files_bucket}/athenaresults/' # from cfn params
wkgrpathena='primary'                                    # Update, if workgroup is different

## Create the Athena connection string.
connection_string = f"awsathena+rest://@{connathena}:{portathena}/{schemaathena}?s3_staging_dir={s3stagingathena}/&work_group={wkgrpathena}"

## Create the Athena SQLAlchemy engine.
engine_athena = create_engine(connection_string, echo=False)
dbathena = SQLDatabase(engine_athena)

gdc = [schemaathena]
print("Connection to Athena database succeeded: ", gdc[0])


### Step 2 - Generate dynamic prompt templates.
Build a consolidated view of the AWS Glue Data Catalog by combining metadata stored for all the databases in pipe-delimited format.

## Code cell 9 ##

In [ ]:
# Generate dynamic prompts to populate the AWS Glue Data Catalog.
# Harvest AWS Glue crawler metadata.

def parse_catalog():
    # Connect to the Data Catalog.
    columns_str = 'database|table|column_name'
    
    # Define the AWS Glue cient.
    glue_client = boto3.client('glue')
    
    for db in gdc:
        response = glue_client.get_tables(DatabaseName =db)
        for tables in response['TableList']:
            # Classification in the response for S3 and other databases is different. Set classification based on the response location.
            if tables['StorageDescriptor']['Location'].startswith('s3'):  classification='s3' 
            else:  classification = tables['Parameters']['classification']
            for columns in tables['StorageDescriptor']['Columns']:
                    dbname,tblname,colname=tables['DatabaseName'],tables['Name'],columns['Name']
                    columns_str = columns_str+f'\n{dbname}|{tblname}|{colname}'
    return columns_str

glue_catalog = parse_catalog()

print(glue_catalog)


### Step 3 - Define two functions.

Define two functions that (1) determine the best data channel to answer the user query, and (2) generate a response to the user's query.

## Code cell 10 ##

In [ ]:
# Define the Bedrock LLM model to use.

bedrock_model = 'amazon.titan-text-lite-v1'

BEDROCK_CLIENT = boto3.client("bedrock-runtime", 'us-east-1')

# Amazon Titan does not require any inference modifiers.
if bedrock_model == 'amazon.titan-text-lite-v1':
    inference_modifier = {}
else:
    inference_modifier = {"temperature":0.0, "max_tokens":50}

llm = BedrockLLM(model_id=bedrock_model, client=BEDROCK_CLIENT, model_kwargs = inference_modifier)

def identify_channel(query):
    prompt_template_titan = """You are a SQL expert. Convert the below natural language question into a valid SQL statement. The schema has the structure below:\n
     """+glue_catalog+""" 
     \n
     Here is the question to be answered:\n
     {query}
     \n
     Provide the SQL query that would retrieve the data based on the natural language request.\n
     
     """
    
    prompt_template = prompt_template_titan
    
    # print(prompt_template)
    
    ## Define prompt 1.
    PROMPT_channel = PromptTemplate(template=prompt_template, input_variables=["query"])
    
    # Define the LLM chain.
    llm_chain = LLMChain(prompt=PROMPT_channel, llm=llm)
    # Run the query and save to generated texts.
    generated_texts = llm_chain.run(query)
    
    # Set the channel from where the query can be answered.
    if 's3' in generated_texts: 
            channel='db'
            db=dbathena
            print("SET database to athena")
    elif 'api' in generated_texts: 
            channel='api'
            print("SET database to weather api")        
    else: raise Exception("User question cannot be answered by any of the channels mentioned in the catalog")
    # print("Step complete. Channel is: ", channel)
    
    return channel, db




## Code cell 11 ##

In [ ]:
def run_query(query):

    channel, db = identify_channel(query) # Call the identify channel function first.

    _DEFAULT_TEMPLATE = """
    Here is a schema of a table:
    <schema>
    {table_info}
    </schema>       
    Run a SQL query to answer the question. Follow this format:
    
    SQLQuery: the correct SQL query. For example: select count ( * )  from s3_library_data where genre = 'Novel'
    SQLResult: the result of the SQL query.
    Answer: convert the SQLResult to a grammatically correct sentence.
    
    Here is question: {input}"""
    
    PROMPT_sql = PromptTemplate(
        input_variables=["table_info","input"], template=_DEFAULT_TEMPLATE
    )

    
    if channel=='db':
        db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=PROMPT_sql, verbose=True, return_intermediate_steps=False)
        response=db_chain.run(query)
    else: raise Exception("Unlisted channel. Check your unified catalog")
    return response

### Step 4 - Run the run_query function.

Running the run_query function, in turn, calls the Langchain SQL database chain to convert 'text to sql', and then runs the query against the source data channel.

The following samples are provided for test runs. Uncomment one query at a time to run it.
## Code cell 12 ##

In [ ]:
# query = """How many books with a genre of Fantasy are in the library?""" 
# query = """Find 3 books in the library with Tarzan in the title?""" 
# query = """How many books by author Stephen King are in the library?""" 
query = """How many total books are there in the library?""" 

response =  run_query(query)
print("----------------------------------------------------------------------")
print(f'SQL and response from user query {query}  \n  {response}')

### Step 5 - Do it yourself.

Review the files in the left file browser. Review code cells 1–11 and pay close attention to the notes and comments.
After updating any variables, rerun the necessary code cells so that a new database is created for Cars. 

The following samples are provided for test runs. Uncomment the query to run it.

Note: You might need to run the next code cell twice if the first time the LLM uses double quotes.

## Code cell DIY ##



In [ ]:
# query = """What car has the highest horsepower in the database?""" 
# query = """Provide the make and price of the cheapest car?""" 
query = """What is the average price of a car in the database?""" 

response =  run_query(query)
print("----------------------------------------------------------------------")
print(f'SQL and response from user query {query}  \n  {response}')